In [66]:
#Importer relevante biblioteker
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

#PARAMETRE

lower = 4999
upper = 6000


#Definer en header, der bliver sendt med GET-request
header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36"
    }


#Sti til Arbejdstilsynets smiley-side
url = "https://websmiley.at.dk/websmiley/advancedsearchform.aspx"

#Send GET-request og hent Arbejdstilsynets smiley-side
r = requests.get(url,headers=header)

#Hvis status er OK parser scriptet indholdet, henter formData og tilføjer det til et dictionary
if r.status_code == 200:
    soup = BeautifulSoup(r.content, "html.parser")
    
    viewstate = soup.select("#__VIEWSTATE")[0]['value']
    eventvalidation = soup.select("#__EVENTVALIDATION")[0]['value']
    
    
    formData = {
        '__EVENTVALIDATION': eventvalidation,
        '__VIEWSTATE': viewstate,
        '__EVENTTARGET': '',
        'companyName': 'Virksomhedens navn',
        'cvrNr': 'CVR-nr.',
        'pNr': 'P-nr.',
        'ddlBrancheGrupper':0,
        'ddlRegioner': 0,
        'ddlSmiley':0,
        'btnDownload': 'Download smileydata'
        
        
    }
    
    
#Send POST-request, hvor den netop hentede formdata sendes med
r = requests.post(url, data=formData, headers=header)



In [76]:
#Konverter responsdata til en string og indlæs i en DataFrame
text = r.text
df = pd.read_csv(StringIO(text),sep=";",keep_default_na=False)

#Filtrer efter fynske virksomheder
fyn = df[(df[" POSTNR"]>lower) & (df[" POSTNR"]<upper)]

roed = fyn[fyn[" SMILEY"] == 'rød']

roed

,CVRNR,PNR,BRANCHEGRUPPE,BRANCHE,NAVN,ADRESSE,POSTNR,POSTDISTRIKT,SMILEY,SMILEY_DATO,REAKTIONS_TYPE,ARBEJDSMILJØPROBLEMER,REAKTIONS_STATUS,Unnamed: 13
1235,10314631,1000055673,Opførelse og nedrivning af byggeri,Gennemførelse af byggeprojekter,AMSTRUP & BAGGESEN ENTREPRISE A/S,"Lumbyvej 11B, 1., th",5000,Odense C,rød,18-12-2022 00:00:00,Strakspåbud,121 Bygherrer,Efterkommet,
1236,10314631,1000055673,Opførelse og nedrivning af byggeri,Gennemførelse af byggeprojekter,AMSTRUP & BAGGESEN ENTREPRISE A/S,"Lumbyvej 11B, 1., th",5000,Odense C,rød,18-12-2022 00:00:00,Kompetence kompleks,121 Bygherrer,Efterkommet,
1813,37118311,1001752114,Nærings- og nydelsesmidler,"Fremstilling af læskedrikke, fremstilling af m...",A/S BRYGGERIET VESTFYEN,Fåborgvej 4,5610,Assens,rød,14-08-2022 00:00:00,Påbud,091 Maskiner og trykbærende udstyr,Efterkommet,
1814,37118311,1001752114,Nærings- og nydelsesmidler,"Fremstilling af læskedrikke, fremstilling af m...",A/S BRYGGERIET VESTFYEN,Fåborgvej 4,5610,Assens,rød,14-08-2022 00:00:00,Kompetence kompleks,091 Maskiner og trykbærende udstyr,Efterkommet,
7765,26259495,1003076140,Engros,Ikke-specialiseret engroshandel,COOP ODENSE NONFOOD CENTER,Hestehaven 51,5260,Odense S,rød,17-11-2022 00:00:00,Strakspåbud,"093 Transport-, løfte- og hejseudstyr samt ele...",Efterkommet,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38515,20394994,1004282653,Metal og maskiner,Fremstilling af øvrige maskiner til specielle ...,UNIK KONSTRUKTION OG DESIGN V/KELD GROTH,Sankt Jørgens Vej 54,5700,Svendborg,rød,02-09-2022 00:00:00,Forbud,094 Fald til lavere niveau,Efterkommet,
40965,35209115,1003309588,Døgninstitutioner og hjemmepleje,Døgninstitutioner for personer med psykiske ha...,3 - Tornbjerggård,Munkehatten 10,5220,Odense SØ,rød,28-04-2022 00:00:00,Påbud,082 Luftkvalitet,Ikke efterkommet,
40966,35209115,1003309588,Døgninstitutioner og hjemmepleje,Døgninstitutioner for personer med psykiske ha...,3 - Tornbjerggård,Munkehatten 10,5220,Odense SØ,rød,28-04-2022 00:00:00,Påbud,082 Luftkvalitet,Ikke efterkommet,
40967,35209115,1003309588,Døgninstitutioner og hjemmepleje,Døgninstitutioner for personer med psykiske ha...,3 - Tornbjerggård,Munkehatten 10,5220,Odense SØ,rød,28-04-2022 00:00:00,Kompetence kompleks,082 Luftkvalitet,Ikke efterkommet,
